<a href="https://colab.research.google.com/github/vibhu18116/IR-Project-Group23-PatentPal/blob/main/Summary/BigPatent_Baseline_Pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pickle
data = pickle.load(open("/content/drive/MyDrive/bigPatentData/data_pik", 'rb'))

In [9]:
import pandas as pd
df = pd.DataFrame(data[400:500])

In [ ]:
!pip install torch==1.2.0 torchvision==0.4.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 25kB/s 
     |████████████████████████████████| 8.8MB 24.1MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.2.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 735.5MB 24kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=d6c5a78aa969f4b21d4d00fcc0a8a23a2cc42862f69e2ff8c89ddfbfa5a24636
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
Successfully built sentence-transformers
ERROR: torchvision 0.4.0+cu92 has requirement torch==1.2.0, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.2.0+cu92
    Uninstalling torch-1.2.0+cu92:
      Successfully uninstalled torch-1.2.0+cu92


In [ ]:
!pip install transformers

In [1]:
!pip install rouge_score

In [11]:
import random
from rouge_score import rouge_scorer

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rouge2', 'rougeL'], use_stemmer=True):
    #Calculate rouge scores
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    
    n = len(pred_summaries)
    
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
    
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

In [ ]:
myfile = open('pegasus-opt.txt', 'w')

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import torch
# src_text = [
#     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
# ]

src_text = df['description'][0]
model_name = 'google/pegasus-big_patent'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest', return_tensors='pt').to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)
# assert tgt_text[0] == "California's largest electricity provider has turned off power to hundreds of thousands of customers."

/home/vibhu_iiitd/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


['An apparatus for the treatment of small quantities of material, the apparatus comprising a container of near- spherical form, the container being mounted on a base which is inclined to the vertical by an angle of between 5  and 20.']


In [ ]:
df['abstract'][0]

'the invention relates to an apparatus for mincing and / or beating , comminuting , chopping , pulverizing , emulsifying , mixing , kneading , whisking and the like of materials , particularly edible materials , such as meat , starchy foods , fruit and vegetables which comprises a near - spherical container , at least one knife mounted for rotation within the container , coupled with safety features to prevent injuries to the operators .'

In [ ]:
df['description'][0]

'the apparatus illustrated in fig1 and 2 has a container 1 , which is of near - spherical form and which has a removable cover 2 , enabling material to be treated to be introduced into the container . the container 1 is mounted on or in a base or frame 12 , by means of two interfitting rings or rims 13 and 14 , ring 13 being integral with base 12 and ring 14 being integral with container 1 . base 12 forms a casing for an electric motor 5 having a drive shaft 4 , and is so constructed that shaft 4 is inclined to the vertical by an angle ω , when the apparatus is located upon a horizontal table t . the inventor conducted a considerable amount of experiments and ascertained that the near - spherical container must be tilted to the optimum angle which is preferably between 5 ° to 20 °. it will be understood that the motor housing encompasses the control circuitry subsequently described in connection with fig1 . container 1 is formed at its base with an inwardly - extending protuberance 3 w

In [ ]:
for i in df['description']:
    src_text = i
    batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest', return_tensors='pt').to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(tgt_text)
    myfile.write("%s\n" % tgt_text[0])
myfile.close()

In [5]:
f = open("pegasus-opt.txt", "r")
pred_summaries = f.readlines()

In [13]:
import numpy as np
scores_pegasus = calc_rouge_scores(pred_summaries, df['abstract'].tolist())

In [14]:
scores_pegasus

{'rouge1': {'f1': 0.36411618641995963,
  'precision': 0.28941894511726557,
  'recall': 0.5772498951793764},
 'rouge2': {'f1': 0.15732418417547966,
  'precision': 0.12402870466732557,
  'recall': 0.25435794525959343},
 'rougeL': {'f1': 0.2570086670250988,
  'precision': 0.20381150084147137,
  'recall': 0.41320167910601263}}